In [71]:
import ast, inspect, itertools, math, numpy as np

ALLOWED_FUNCTIONS = {'itertools', 'numpy', 'np', 'math', 'functools', 'collections', 'random'}
DISALLOWED = { '__import__(', 'breakpoint(', 'compile(', 'open(', 'dir(', 'eval(', 'exec(', 'globals(',
              'input(', 'repr(', 'savetxt(', 'loadtxt(', 'genfromtxt(', 'fromfile(', 'tofile(', 'frombuffer(',
              'save(', 'savez(', 'savez_compressed(', 'load(', 'savetxtas', 'loadtxtas', 'genfromtxtas', 
              'fromfileas', 'tofileas', 'frombufferas', 'saveas', 'savezas', 'savez_compressedas',
              'loadas', '=__import__\n', '=breakpoint\n', '=compile\n', '=open\n', '=dir\n', '=eval\n', '=exec\n', '=globals\n',
              '=input\n', '=repr\n', '=savetxt\n', '=loadtxt\n', '=genfromtxt\n', '=fromfile\n', '=tofile\n', '=frombuffer\n',
              '=save\n', '=savez\n', '=savez_compressed\n', '=load\n', '=__import__,', '=breakpoint,', '=compile,', '=open,', '=dir,',
              '=eval,', '=exec,', '=globals,', '=input,', '=repr,', '=savetxt,', '=loadtxt,', '=genfromtxt,', '=fromfile,',
              '=tofile,', '=frombuffer,', '=save,', '=savez,', '=savez_compressed,', '=load,'}

In [72]:
def my_function():
    import numpy as np
    from numpy import sqrt, exp
    x = list([1,2,3,4])

    return sqrt(2)

is_function_safe(my_function)

True

In [59]:
def is_function_safe(func):
    source = inspect.getsource(func)
    parsed = ast.parse(source)

    imported_packages = [
        node.module for node in ast.walk(parsed)
        if isinstance(node, ast.ImportFrom)
    ]

    if any(pkg not in ALLOWED_FUNCTIONS for pkg in imported_packages):
        print("Import form")
        print([pkg for pkg in imported_packages if pkg not in ALLOWED_FUNCTIONS])
        return False
    
    imported_packages = [
        node.names for node in ast.walk(parsed) if isinstance(node, ast.Import)
    ]
    imported_packages = [name.name for node in imported_packages for name in node]

    if any(pkg not in ALLOWED_FUNCTIONS for pkg in imported_packages):
        print("Import")
        print([pkg for pkg in imported_packages if pkg not in ALLOWED_FUNCTIONS])
        return False
        
    if any (banned in source.replace(" ", "") for banned in DISALLOWED):
        print("Function")
        print([banned for banned in DISALLOWED if banned in source.replace(" ", "")])
        return False
    
    return True

In [60]:
is_function_safe(my_function)

True

In [61]:
def priority1(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  pair_count = len(set(itertools.combinations(v, 2)))
  return pair_count / (n * (n - 1) / 2)


def priority2(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  v = sorted(v)
  pair_count = sum(1 for i in range(1, n) if v[i] != v[i-1])
  return pair_count / n


def priority3(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  unique_values = np.unique(v)
  pair_count = sum(1 for i in range(n) for j in range(i+1, n) if v[i] != v[j])
  diversity = len(unique_values) / n
  return pair_count / n + diversity


def priority4(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  unique_elements = len(set(v))
  pair_count = sum(1 for i in range(n) for j in range(i+1, n) if v[i] != v[j])
  return (unique_elements * pair_count) / (n * (n-1))

def priority5(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  # This function is identical to `priority_v2`; we're only adding it to maintain the sequence of function names
  return priority_v2(v, n)


def priority6(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  count = 0
  for i in range(n):
    for j in range(i+1, n):
      if v[i] != v[j]:
        count += 1
  return count / n


def priority7(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  # Convert vector to a bitmask by inverting each 3-valued entry and joining them as a single binary number
  bitmask = sum(2**i for i in range(n) if v[i] != 0)
  # Calculate the number of 1s in the bitmask to get the pair count
  pair_count = np.popcount(bitmask)
  return pair_count / n


def priority8(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  unique_pairs = len(set(list(itertools.combinations(v, 2))))
  return unique_pairs / np.math.comb(n, 2)

def priority9(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
    The cap set will be constructed by adding vectors that do not create a line in order by priority.
  """
  unique_pairs = open('test')
  return unique_pairs / np.math.comb(n, 2)

def priority10(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
      The cap set will be constructed by adding vectors that do not create a line in order by priority.
      This version checks for collinearity by calculating the determinant of the 3x3 matrix made from the vectors.
  """
  # Calculate the dot product of each pair of vectors
  dot_products = np.outer(v, v)[::-1].sum(axis=0)

  # Add a small epsilon to avoid singularity
  epsilon = 1e-8
  determinant = np.linalg.det(np.vstack((v, dot_products, np.ones(n))).T + epsilon * np.eye(n + 1))

  # Normalize the determinant to a range of [0, 1]
  return 1.0 - np.abs(determinant) / (np.linalg.det(np.eye(n + 1)) ** 2)

def priority11(v: tuple[int, ...], n: int) -> float:
  """Returns the priority, as a floating point number, of the vector `v` of length `n`. The vector 'v' is a tuple of values in {0,1,2}.
      The cap set will be constructed by adding vectors that do not create a line in order by priority.
      This version uses a simple heuristic to prioritize vectors with fewer lines.
  """
  lines = set()
  for i in range(n):
    for j in range(i+1, n):
      if np.sum(np.array(v[i:j+1]) == np.array(v[i:j+1]).sum(axis=0)) == 3:
        lines.add((i, j))
  if lines:
    return len(lines)
  else:
    return 0.0

In [62]:
%timeit is_function_safe(priority1)
%timeit is_function_safe(priority2)
%timeit is_function_safe(priority3)
%timeit is_function_safe(priority4)
%timeit is_function_safe(priority5)
%timeit is_function_safe(priority6)
%timeit is_function_safe(priority7)
%timeit is_function_safe(priority8)
%timeit is_function_safe(priority9)
%timeit is_function_safe(priority10)
%timeit is_function_safe(priority11)


227 µs ± 8.77 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
257 µs ± 2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


KeyboardInterrupt: 

In [63]:
print(is_function_safe(priority1))
print(is_function_safe(priority2))
print(is_function_safe(priority3))
print(is_function_safe(priority4))
print(is_function_safe(priority5))
print(is_function_safe(priority6))
print(is_function_safe(priority7))
print(is_function_safe(priority8))
print(is_function_safe(priority9))
print(is_function_safe(priority10))
print(is_function_safe(priority11))

True
True
True
True
True
True
True
True
Function
['open(']
False
True
True


In [64]:
def priority_v2(v: tuple[int, ...], n: int) -> float:
    """Improves the priority calculation by considering the distribution of 0's, 1's, and 2's in the vector 'v'.
    The priority is inversely proportional to the number of occurrences of each unique value in 'v'.
    """
    unique_values = np.unique(v)
    counts = [list(v).count(val) for val in unique_values]
    if 1 in counts:
        priority = np.mean([1/c for c in counts if c != 1])
    else:
        priority = 1.0
    return priority

is_function_safe(priority_v2)

True

In [65]:
def priority_v2(v: tuple[int, ...], n: int) -> float:
  """Improved version of `priority_v1`.
  """
  sums, freqs = set(), {}
  for _ in itertools.combinations(v, 3):
    sum_ = sum(x for x in _)
    sums.add(sum_)
    if sum_ not in freqs:
      freqs[sum_] = 0
    freqs[sum_] += 1
  return len(sums) / (n * (n - 1) * (n - 2)) + len(freqs)

is_function_safe(priority_v2)

True

In [66]:
def priority_v2(v: tuple[int, ...], n: int) -> float:
  """Improved version of `priority_v1`.
  """
  sums, freqs = set(), {}
  for _ in itertools.combinations(v, 3):
    sum_ = sum(x for x in _)
    sums.add(sum_)
    if sum_ not in freqs:
      freqs[sum_] = 0
    freqs[sum_] += 1
  return len(sums) / (n * (n - 1) * (n - 2)) + len(freqs)

is_function_safe(priority_v2)

True

In [67]:
def priority_v3(v: tuple[int, ...], n: int) -> float:
  """Further improved version of `priority_v1`.
  """
  unique_counts = np.bincount(v, minlength=3)
  two_counts = np.bincount(unique_counts, minlength=2)
  three_counts = np.bincount(unique_counts, minlength=3)

  # Calculate the count of unique vectors and vectors with 2 or 3 occurrences
  unique_counts_nonzero = unique_counts[unique_counts > 0].sum()
  two_counts_nonzero = two_counts[two_counts > 0].sum()

  # Calculate the number of vectors with 3 occurrences
  three_counts_nonzero = three_counts[three_counts > 0].sum()

  # Penalty for vectors with 3 occurrences
  penalty = 3 * three_counts_nonzero

  # Penalty for vectors with 2 occurrences, considering their multiplicity
  two_counts_penalty = sum(i * (i - 1) for i in two_counts[two_counts > 0])

  # Return the priority value
  return np.prod(unique_counts) * (n - unique_counts_nonzero) * len(two_counts[two_counts > 0]) * np.exp(-penalty - 0.5 * two_counts_penalty)

is_function_safe(priority_v3)

True

In [68]:
def priority_v2(v: tuple[int, ...], n: int) -> float:
  """Improved version of `priority_v1`.
  This version considers the distance between consecutive points.
  """
  diff = np.abs(np.roll(v, 1) - v)
  distances = np.sum(diff)
  direction_changes = (diff[1:] != diff[:-1]).sum()
  return distances * n + (n - 1) * (distances / (n - 1)) * direction_changes

is_function_safe(priority_v2)

True

In [69]:
def priority_v3(v: tuple[int, ...], n: int) -> float:
    """
    Improved version of `priority_v2`. This version uses the Counter from collections module to improve efficiency.
    """
    from collections import Counter
    value_counter = Counter(v)
    return len(set(value_counter.elements()) & {0, 1, 2})

is_function_safe(priority_v3)

True

In [70]:
def priority_v3(v: tuple[int, ...], n: int) -> float:
    """Improved version of `priority_v2`.
    Adds a directional symmetry penalty to discourage vectors that are too similar.
    """
    value_counts = {0: v.count(0), 1: v.count(1), 2: v.count(2)}

    # Soft zero penalty with reduced impact
    zero_penalty = (value_counts[0] / n) ** 0.5

    # Entropy-like balance score
    balance_score = 1 - sum((count/n) * np.log(count/n + 1e-10) for count in value_counts.values())

    # Distribution bonus: reward vectors with more even spread
    distribution_bonus = 1 - np.std([count/n for count in value_counts.values()])

    # Symmetry penalty: discourage vectors that are too directionally uniform
    directional_symmetry = max(
        abs(sum(1 if x == i else -1 if x == (i+1)%3 else 0 for x in v)) / n
        for i in range(3)
    )
    symmetry_penalty = directional_symmetry

    # Combine metrics with different weighting
    return balance_score - 0.5 * zero_penalty + 0.2 * distribution_bonus - 0.3 * symmetry_penalty

is_function_safe(priority_v3)

True

In [73]:
def priority_v2(v: tuple[int, ...], n: int) -> float:
  """Improved version of `priority_v1`.
  """
  unique_triangles = len(set(tuple(sorted(set(x))) for x in itertools.combinations(v, 3)))
  dense_triangles = sum(len(set(tuple(sorted(set(x))) for x in itertools.combinations(v[i:i+3], 3))) for i in range(n-2)) / (n-2)
  max_triangles = n * (n - 1) * (n - 2) / 6
  # Adjust the bias based on the number of unique values in the vector
  unique_values = len(set(v))
  if unique_values < 3:
    bias = np.sqrt(dense_triangles)
  else:
    bias = np.sqrt(dense_triangles * (3 - unique_values) / (3 * (n - 1)))
  # Small change: calculate the average of smallest circle radii of triangles
  import scipy.spatial.distance as spd
  all_distances = []
  for i in range(n-2):
    for j in range(i+1, n-1):
      for k in range(j+1, n):
        triangle = np.array([v[i], v[j], v[k]])
        dists = spd.pdist(triangle)
        radius = dists.min() / 2
        all_distances.append(radius)
  avg_radius = np.mean(all_distances)
  # Normalize the result between 0 and 1
  priority = (unique_triangles / max_triangles + bias + avg_radius) / 3
  return priority

is_function_safe(priority_v2)

Import
['scipy.spatial.distance']


False